In [1]:
#import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import tensorflow as tf
import time
from skimage import io
from skimage.transform import resize
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential, Model
from napari.layers import Image, Labels
from csbdeep.utils import normalize
from magicgui import magicgui
import napari

In [2]:
import nrrd
import nibabel as nib
import pandas as pd

ERROR:pydicom.pixels.utils:No module named 'pylibjpeg'
Traceback (most recent call last):
  File "/Users/stephanehess/Documents/CAS_AML/venv/lib/python3.12/site-packages/pydicom/pixels/utils.py", line 1274, in _passes_version_check
    module = importlib.import_module(package_name, "__version__")
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/python@3.12/3.12.5/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1324, in _find_and_load_unlocked
ModuleNotFoundError: No module named 'pylibjpeg'
ERROR:pydicom.pixels.utils:No module named 'pylibjpeg'
Traceback (most recent call last):
 

In [3]:
os.listdir()

['convpaint_params.yml',
 'notebook_M2D3_1.ipynb',
 'output.nii.gz',
 '.DS_Store',
 'requirements.txt',
 'notebook_M2D2_1.ipynb',
 'notebook_M2D3_2.ipynb',
 'Untitled.ipynb',
 'DIRU_2022_04_dog_03_E_nc',
 'DIRU_2022_04_dog_03_E_30c',
 'segmentation_transposed_tryout.png',
 'notebook_M2D2_2.ipynb',
 'false_segmentation.joblib',
 'notebook_M1D3_2.ipynb',
 'meniscus_segment_tryout.joblib',
 'data_inspection.ipynb',
 '6-13_snake.jpg',
 'screenshot_segmentation_tryout.jpg',
 '.ipynb_checkpoints',
 'notebook_M1D3_1.ipynb',
 'notebook_M1D4.ipynb',
 'false_segmentation.png',
 'segmentation_tryout.tif',
 'napari_pixel_classifier_rgb.ipynb',
 'notebook_M1D1_D2.ipynb',
 'notebook_M2D1.ipynb']

### Load data:

In [7]:
file_path = 'DIRU_2022_04_dog_03_E_nc/medialis/03_E_nc_med_Segmentation.seg.nrrd'
#file_path = 'DIRU_2022_04_dog_03_E_30c/medialis/03_E_30c_med_Segmentation.seg.nrrd'

In [8]:
# Load the NRRD file
data, header = nrrd.read(file_path)

# Create a NIfTI image
img = nib.Nifti1Image(data, np.eye(4))

# Save the image in NIfTI format
nib.save(img, 'output.nii.gz')

In [9]:
data = img.get_fdata()

In [10]:
data.shape

(4, 560, 560, 484)

### It turns out channel 0 is the one that has meniscus as well as background signal:

In [11]:
data_ex = data[0, :, :, :]
#data_ex = data[ :, :, :]

In [12]:
data_ex.shape

(560, 560, 484)

### Find slices where signal is visible:

In [13]:
indices = []
max_vals = []
sum_vals = []
# Loop through dimension 
for ix in range(0, data_ex.shape[0]):
# Get maximum pixel value: 
    max_val = np.max(data_ex[ix, :, :])
    sum_val = data_ex[ix, :, :].sum()
# Only choose indices of sclices with pixel-value bigger than 0,
    # i.e. where a signal is present:
    if max_val > 0:
        #print(ix)
        indices.append(ix)
        max_vals.append(max_val)
        sum_vals.append(sum_val)
        
indices_pixel_vals = pd.DataFrame({'index_value': indices, 'maximum': max_vals, 
             'pixel_sum': sum_vals})

indices_pixel_vals.head()


,index_value,maximum,pixel_sum
0,138,2.0,18.0
1,139,2.0,22.0
2,140,2.0,26.0
3,141,2.0,30.0
4,142,2.0,34.0


### Have a look at where the signal is the strongest:

In [ ]:
indices_pixel_vals.plot('index_value', 'maximum')

In [ ]:
# The signal appears for index-values of first dimension bigger than 250:
sel_bools = indices_pixel_vals.index_value > 250
indices_pixel_vals.loc[sel_bools]

In [ ]:
sel_bools = indices_pixel_vals.maximum > 2
selected_indices = indices_pixel_vals.loc[sel_bools]

In [ ]:
selected_indices.head()

In [ ]:
selected_indices.tail()

### Select a few indices, where pictures will show:

In [ ]:
indices_to_plot = list(np.arange(240, 440, 10))

In [ ]:
np.max(data_ex[:, 270, :])

In [ ]:
for ix in indices_to_plot:
    
    #slice_0 = data_ex[ix, :, :]
    slice_0 = data_ex[:, ix, :]
    #rotated_img = ndimage.rotate(slice_0, 5*60)
    print(ix)
    # Plot the slice
    plt.imshow(slice_0, cmap='gray', origin='lower')
    plt.title('Example')
    plt.show()

Loop through the slices of the meniscus: 

In [ ]:
for ix in range(267,287):
    
    #slice_0 = data_ex[ix, :, :]
    slice_0 = data_ex[:, ix, :]
    #rotated_img = ndimage.rotate(slice_0, 5*60)
    print(ix)
    # Plot the slice
    plt.imshow(slice_0, cmap='gray', origin='lower')
    plt.title('Example')
    plt.show()

### Conclusion:

For a first try, let's use images with indices between 270 and 285 along the first dimenstion.

In [ ]:
data_ex[:, 281, :]

In [ ]:
np.unique(data_ex[:, 281, :], return_counts=True)

In [ ]:
plt.imshow(data_ex[:, 281, :], cmap='gray')
plt.title('Example')
plt.show()

In [ ]:
example_slice = data_ex[:, 281, :].copy()

In [ ]:
np.unique(example_slice)

In [ ]:
plt.imshow(example_slice, cmap='gray')
plt.title('Example')
plt.show()

In [ ]:
example_slice_copy = example_slice.copy()
np.unique(example_slice_copy)

In [ ]:
example_slice_copy[example_slice_copy==3] = 0

In [ ]:
np.unique(example_slice_copy)

In [ ]:
example_slice_copy[example_slice_copy==2] = 1

In [ ]:
np.unique(example_slice_copy)

In [ ]:
plt.imshow(example_slice_copy, cmap='gray')
plt.title('Example')
plt.show()

In [ ]:
slice_train = example_slice_copy

In [ ]:
example_slice_copy2 = example_slice.copy()

In [ ]:
np.unique(example_slice_copy2)

In [ ]:
plt.imshow(example_slice_copy2, cmap='gray')
plt.title('Example')
plt.show()

In [ ]:
example_slice_copy2[example_slice_copy2!=3] = 0
np.unique(example_slice_copy2)

In [ ]:
example_slice_copy2[example_slice_copy2==3] = 1
np.unique(example_slice_copy2)

In [ ]:
plt.imshow(example_slice_copy2, cmap='gray')
plt.title('Example')
plt.show()

In [ ]:
labels = example_slice_copy2

In [ ]:
example_slice_2 = data_ex[:, 280, :].copy()
np.unique(example_slice_2)

In [ ]:
example_slice_2[example_slice_2==3] = 0
np.unique(example_slice_2)

In [ ]:
example_slice_2[example_slice_2==2] = 1
np.unique(example_slice_2)

In [ ]:
plt.imshow(example_slice_2, cmap='gray')
plt.title('Example')
plt.show()

In [ ]:
slice_test = example_slice_2

input_image_stacked = np.expand_dims(img, axis=-1)
#as it works only with 3 input channels: stack nuclear channel
stacked_img = np.stack((img_scaled,)*3, axis=2)
stacked_img = np.squeeze(stacked_img)
stacked_img = stacked_img.astype(np.float32)

## Load some image data as `np.array`
Only works with 3-channel data. If it's grayscale data, just stack the image 3 times using the function `grayscale_to_rgb()`.

In [ ]:
def grayscale_to_rgb(grayscale_img):
    img = np.expand_dims(grayscale_img, axis=-1)
    stacked_img = np.stack((img,)*3, axis=2)
    stacked_img = np.squeeze(stacked_img)
    return stacked_img

In [ ]:
img = grayscale_to_rgb(slice_train)
#img = grayscale_to_rgb(slice_train)
#img = grayscale_to_rgb(slice_test_t)

import urllib.request
from PIL import Image
urllib.request.urlretrieve('https://upload.wikimedia.org/wikipedia/commons/5/58/6-13_snake.jpg', '6-13_snake.jpg')
img_pil = Image.open('6-13_snake.jpg')
img = np.array(img_pil)
plt.imshow(img)
plt.show()

In [ ]:
img.shape

In [ ]:
slice_train.shape

 ## Extract the features using convolutional filters extracted from a VGG16

In [ ]:
#create the classifier
clf = RandomForestClassifier(random_state=0)

# create feature extractor and apply 
shapes = []
for i in [1,2,4]:
    shapes.append((int(img.shape[0]/i),(int(img.shape[1]/i))))
print("Image input scales:")
print(shapes)
models = init_VGG16_pyramid(shapes)
features = fd_VGG16_pyramid(img,models,shapes)
# plot example filter response
plt.imshow(features[:,:,100])
plt.show()

In [ ]:
plt.imshow(features[:,:,120])
plt.show()

In [ ]:
features.shape

In [ ]:
labels.shape

In [ ]:
labels

In [ ]:
# Flatten the image (combine first two dimensions: 560 * 484 = 271,040 pixels)
X = features.reshape(-1, 192)  # Shape: (271040, 192)

In [ ]:
# Flatten the labels
y = labels.ravel()  # Shape: (271040,)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
clf.fit(X, y)

In [ ]:
predictions = clf.predict(X)

In [ ]:
# Assuming `predictions` is the result from your classifier, and it's a 1D array
# of shape (271040,), corresponding to (560 * 484)
predictions_reshaped = predictions.reshape(560, 484)

# Plot the reshaped predictions
plt.imshow(predictions_reshaped, cmap='gray')
plt.title('Predicted Segmentation')
plt.colorbar()  # Optional: Add a colorbar
plt.show()

In [ ]:
slice_test

In [ ]:
plt.imshow(slice_test, cmap='gray')
plt.colorbar()  # Optional: Add a colorbar
plt.show()

In [ ]:
img_test = grayscale_to_rgb(slice_test)

In [ ]:
# create feature extractor and apply 
shapes = []
for i in [1,2,4]:
    shapes.append((int(img_test.shape[0]/i),(int(img_test.shape[1]/i))))
print("Image input scales:")
print(shapes)
models = init_VGG16_pyramid(shapes)
features_test = fd_VGG16_pyramid(img_test,models,shapes)
# plot example filter response
plt.imshow(features_test[:,:,100])
plt.show()

In [ ]:
features_test.shape

In [ ]:
# Flatten the image (combine first two dimensions: 560 * 484 = 271,040 pixels)
X_test = features_test.reshape(-1, 192)  # Shape: (271040, 192)

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
# Assuming `predictions` is the result from your classifier, and it's a 1D array
# of shape (271040,), corresponding to (560 * 484)
predictions_reshaped = predictions.reshape(560, 484)

# Plot the reshaped predictions
plt.imshow(predictions_reshaped, cmap='gray')
plt.title('Predicted Segmentation')
plt.colorbar()  # Optional: Add a colorbar
plt.show()

## Annotate some pixels and train the classifier!
Labels:  
0 : Not annotated (transparent)  
1 : Foreground (yellow)  
2 : Background (red)  

In [ ]:
viewer = napari.view_image(img, name="My Image")
viewer.window.add_dock_widget(clf_widget)
prediction_layer = viewer.add_image(np.zeros_like(img[:,:,0]),colormap='PiYG',contrast_limits=(0, 1))
prediction_layer.visible = False
prediction_layer.name = "Classifier prediction"
labels_layer = viewer.add_labels(np.zeros_like(img[:,:,0]).astype(int), seed = 0)
napari.run()

In [ ]:
# plot the final prediction image
plt.imshow(prediction_layer.data)

In [ ]:
#plot the drawn annotations
plt.imshow(labels_layer.data)

## In depth description of classifier
### Feature extraction using VGG16 transfer learning
VGG16 is an is a convolutional neural network for image recognition proposed by Simonyan and Zisserman. A model trained on the ImageNet Dataset is downloaded trough Keras (TensorFlow backend). Training is disabled and the VGG16 network is cut off before the first Pooling layer. In consequence the output layer preserves the spatial resolution of the input image, resulting in a feature vector of length 64 for each pixel. As the network has been trained on the task of general object detection on ImageNet, the filters in the first layers capture low level features such as edges or textures that can be used for characterising objects such as nuclei. The relatively small filter size (3\*3px) VGG16 introduced (previous state of the art was 11\*11px and 7\*7px), and the discard-
ing of the deeper levels means that only the immediate neighbourhood is taken into account when calculating the features for each pixel. This leads to a noisy prediction, and loss of information of larger scale features. We
found that the noisy prediction and loss of information of larger scale features can be counteracted by inputting
the image at multiple scales, constructing a *featurized image pyramid*.
### Construction of a featurized image pyramid
CNNs using different scales/resolutions of the input image are used in popular segmentation models like UNets, and across different domains (e.g. self driving). Although there exist faster approaches of providing differently scaled data to a neural networks, (e.g.by calculating a *pyramidal feature hierarchy* or by using *Feature Pyramid Networks*, see Lin et al.for an overview), these strategies are not applicable with the transfer learning approach relied on here. Four scalings (1/1, 1/2, 1/4, 1/8) are applied to the input image. The scaled images are then passed separately trough the modified VGG16 model. The outputs are scaled up to match the original spatial  resolution and then concatenated, resulting in 4\*64 = 256 features per pixel. No smaller scales were used as adding them did not improve segmentation results, they should however be considered if working with higher resolution images or different microscope optics (e.g. magnification over 40x).